In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [8]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = { "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]) }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [9]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc5c1c410f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [10]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
Tensor("strided_slice:0", shape=(), dtype=int32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.09660675 0.09569006 0.10687497 0.10159525 0.09333061 0.09744369
  0.09895156 0.10421658 0.10372987 0.10156067]
 [0.099609   0.10686946 0.0955205  0.10445505 0.10633792 0.10070583
  0.09860536 0.09040209 0.10207797 0.09541681]
 [0.11175021 0.08968595 0.09979179 0.09946194 0.1029023  0.1100158
  0.08586155 0.10402983 0.10121627 0.09528439]
 [0.09606198 0.10194147 0.09761177 0.10528671 0.0982826  0.09676836
  0.09960251 0.10278952 0.103978   0.09767704]
 [0.09946147 0.10010791 0.09614763 0.09845769 0.09768738 0.0

INFO:tensorflow:loss = 2.3033838, step = 2
INFO:tensorflow:probabilities = [[0.09954388 0.0938283  0.10746527 0.10072529 0.09537387 0.10144209
  0.09396847 0.10528424 0.10539259 0.09697599]
 [0.09898101 0.09498455 0.11112022 0.11097983 0.09421773 0.08929006
  0.08796988 0.10092793 0.10438555 0.10714319]
 [0.09520119 0.09755535 0.09759527 0.11416289 0.09890528 0.09364334
  0.09603503 0.09871943 0.11025222 0.09793002]
 [0.09855692 0.09220112 0.10206963 0.10811982 0.0897681  0.10127596
  0.09627296 0.10274785 0.11706985 0.09191781]
 [0.09949988 0.09463929 0.09990048 0.10085754 0.09298375 0.10690715
  0.10058543 0.10014342 0.10718596 0.09729708]
 [0.10277797 0.09571213 0.10310189 0.09993488 0.09854323 0.09919512
  0.09436031 0.10348648 0.10322639 0.09966156]
 [0.10616305 0.09027036 0.09714372 0.11097218 0.08575755 0.10238502
  0.09538128 0.1034788  0.11046958 0.09797849]
 [0.09931001 0.09479819 0.11715566 0.1094886  0.1056952  0.09985279
  0.08402071 0.1020164  0.09642918 0.09123322]
 [0.1

INFO:tensorflow:global_step/sec: 110.442
INFO:tensorflow:probabilities = [[0.10933682 0.08514451 0.10253329 0.10646157 0.10364604 0.09575026
  0.09846413 0.10242325 0.10257499 0.09366506]
 [0.10259476 0.09550782 0.10363806 0.10675281 0.09724388 0.09700891
  0.09852328 0.10567681 0.097628   0.09542566]
 [0.10100304 0.09836846 0.09831278 0.10323804 0.09093868 0.10762361
  0.09316021 0.10266931 0.10473006 0.09995583]
 [0.10888515 0.09386176 0.10356206 0.09141082 0.09969694 0.10326715
  0.09510308 0.10303063 0.10276616 0.09841631]
 [0.10587806 0.09738699 0.10480833 0.10340384 0.09233491 0.09850638
  0.09682733 0.09950382 0.10213576 0.09921461]
 [0.09642174 0.09827429 0.10566873 0.09953231 0.09716991 0.09428171
  0.09563468 0.10655258 0.11038192 0.09608216]
 [0.09985986 0.09603426 0.10180496 0.09920003 0.09328186 0.09739505
  0.09355319 0.10911755 0.10225794 0.10749526]
 [0.10573959 0.09689183 0.10103311 0.09995461 0.10226015 0.10235824
  0.09594306 0.10311534 0.10414901 0.08855509]
 [0.107

INFO:tensorflow:loss = 2.2958126, step = 102 (0.912 sec)
INFO:tensorflow:probabilities = [[0.10367058 0.08673725 0.10558825 0.09732837 0.09770062 0.09664866
  0.0921479  0.10785944 0.09899849 0.11332041]
 [0.11302051 0.09364562 0.10292909 0.09742077 0.10171733 0.09581217
  0.09887195 0.10716856 0.09726901 0.09214497]
 [0.10770777 0.0923526  0.10841209 0.10598888 0.09608741 0.10181717
  0.09404534 0.0940105  0.09754793 0.10203034]
 [0.0931088  0.10422159 0.09706762 0.10533902 0.10150584 0.09517628
  0.10444067 0.10244122 0.10467749 0.09202141]
 [0.10186432 0.09633456 0.10526323 0.1012606  0.10096861 0.09986835
  0.10116861 0.09939142 0.09740112 0.09647916]
 [0.10668223 0.09840344 0.10412487 0.09595375 0.09220288 0.1040967
  0.09313698 0.09916174 0.10430159 0.10193593]
 [0.09791052 0.10388789 0.10507278 0.10192346 0.08805133 0.09861645
  0.09133756 0.10187869 0.12325592 0.08806535]
 [0.10441589 0.1012937  0.10785501 0.09870684 0.0916926  0.10257068
  0.09749813 0.10174887 0.10124553 0.09

INFO:tensorflow:global_step/sec: 127.029
INFO:tensorflow:probabilities = [[0.09972785 0.09324764 0.10782202 0.10091773 0.09428047 0.09664827
  0.09927367 0.10108504 0.1037431  0.10325417]
 [0.11339106 0.09253516 0.10850064 0.10322867 0.09746924 0.0983793
  0.09892885 0.10093553 0.09573115 0.0909004 ]
 [0.10377704 0.09544265 0.0978455  0.1042584  0.09693909 0.09694076
  0.09308207 0.1108979  0.10438564 0.09643099]
 [0.0989729  0.09775381 0.10037128 0.10197834 0.09863929 0.10309164
  0.10224311 0.10266897 0.10002838 0.09425225]
 [0.10740402 0.09698839 0.09151217 0.10248534 0.09254751 0.10666267
  0.10122044 0.09791645 0.10935243 0.0939106 ]
 [0.10808659 0.08644319 0.09616014 0.11034037 0.0944306  0.10660464
  0.09139885 0.10408276 0.10481783 0.09763502]
 [0.10823014 0.09455997 0.09799078 0.09949323 0.09511525 0.10835637
  0.09137912 0.10558378 0.10651989 0.09277147]
 [0.10340417 0.0909527  0.10069577 0.10144245 0.09592801 0.10001475
  0.09092636 0.10867599 0.1118146  0.0961452 ]
 [0.0991

INFO:tensorflow:loss = 2.2773666, step = 202 (0.782 sec)


KeyboardInterrupt: 

In [20]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-03-12-15:09:52
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-03-12-15:09:53
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9718, global_step = 20000, loss = 0.09464826
{'accuracy': 0.9718, 'loss': 0.09464826, 'global_step': 20000}
